In [1]:
import os, sys
sys.path.append('../src')
sys.path.append('../src/utils/')
from spiking_dataloader import WISDM_spiking_dataloader, WisdmDatasetParser
from AHPC_network.AHPC_network import AHPCnetwork
from output_process import OutputProcess

In [2]:
dataset = WisdmDatasetParser('../data/data_watch_subset_0_40.npz', norm=None, class_sublset='custom', subset_list=[0, 2, 6, 8, 9, 14, 17])

(6,)
(6,)
ytrain shape (55404, 18)
yval shape (18468, 18)
ytest shape (18469, 18)
num classes train dataset: 7 occurrences of each class:[3127 2855 3102 3047 3150 2973 3213]
num classes eval dataset: 7 occurrences of each class:[1035  935 1122  996 1110 1007 1077]
num classes test dataset: 7 occurrences of each class:[1046  900 1046 1036 1076  982 1054]


In [3]:
num_samples = 200

In [4]:
val_set = dataset.get_validation_set(subset=num_samples)
val_set[0].shape

(200, 6, 40)

In [5]:
spiking_loader = WISDM_spiking_dataloader(val_set , net_delay=6)

In [6]:
net = AHPCnetwork('/home/franzhd/snntorch_network/nni_experiments/AHPC_7_best_no_encoder/results/1it5cqg3/trials/wZCws/Trained/network_best.npz')
net.print_state_variables()

leaky1 betas:0.3261461853981018
leaky1 vth:1.9845956563949585
leaky2 betas:0.8115220665931702
leaky2 vth:1.0197168588638306
recurrent betas:0.5234979391098022
leaky3 betas:[1.]
leaky3 vth:0.6436737179756165
###network state variables###
leaky1_betas: 0.6738538146018982
leaky1_vth: 1.9845956563949585
leaky2_betas: 0.18847793340682983
leaky2_vth: 1.0197168588638306
leaky3_betas: [0.]
leaky3_vth: 0.6436737179756165



In [7]:
net.print_weights()

###network weights###
linear1_w: [[ 1.10921547e-01  2.90248722e-01 -1.49957642e-01 -8.60216171e-02  5.61085269e-02 -8.76850858e-02]
 [-1.58577152e-02 -5.37142381e-02 -1.23609699e-01  2.11566359e-01 -6.84263408e-02 -2.34918609e-01]
 [-1.01287872e-01  1.23196363e-01  6.46381453e-02  4.36401181e-02  5.93417790e-03 -1.79890152e-02]
 [-1.24119669e-01  1.39249712e-01 -5.09018674e-02 -1.24979146e-01 -1.00728020e-01 -1.63273104e-02]
 [-9.98368785e-02 -7.52779692e-02 -5.75715564e-02  1.31169647e-01  4.36347872e-02  1.27185181e-01]
 [-7.95362443e-02  1.18180633e-01  4.22406644e-02 -1.00603618e-01 -1.15234315e-01 -2.92968184e-01]
 [ 3.82029749e-02 -1.20867535e-01  7.50379041e-02 -9.27442312e-02  7.12862844e-03 -1.84466317e-01]
 [-3.37598026e-02  1.51065439e-01 -1.05463319e-01  3.60094637e-01 -8.88874456e-02  1.68343186e-01]
 [ 4.72156405e-02 -1.18580461e-01  5.06362095e-02 -1.21745197e-02 -3.32989901e-01 -4.44362015e-01]
 [-1.64990537e-02  7.86364749e-02 -1.35770336e-01  9.35193598e-02 -5.4934594

In [8]:
out_sink = OutputProcess(7,num_samples,40, 6)

In [9]:
spiking_loader.data_out.connect(net.a_in)
net.s_out.connect(out_sink.spikes_in)
out_sink.label_in.connect_from(spiking_loader.label_out)

In [10]:

from lava.magma.core.run_conditions import RunSteps
from lava.magma.core.run_configs import Loihi1SimCfg, Loihi2SimCfg
import numpy as np
from tqdm import tqdm

for i in tqdm(range(num_samples)):
        out_sink.run(condition=RunSteps(num_steps=46),
                        run_cfg=Loihi1SimCfg(select_sub_proc_model=True,
                        select_tag='floating_pt'))
        net.reset_hidden_state()

ground_truth = out_sink.gt_labels.get().astype(np.int32)
predictions = out_sink.pred_labels.get().astype(np.int32)

# Stop the execution
out_sink.stop()

accuracy = np.sum(ground_truth[:-1]==predictions[1:])/ground_truth.size * 100

print(f"\nGround truth: {ground_truth[:-1]}\n"
      f"Predictions : {predictions[1:]}\n"
      f"Accuracy    : {accuracy}")



  0%|          | 0/200 [00:00<?, ?it/s]

leaky1 dv:1
leaky1 du:0.6738538146018982
leaky1 vth:1.9845956563949585
leaky2 dv:1
leaky2 du:0.18847793340682983
leaky2 vth:1.0197168588638306
recurrent dv:1
recurrent du:0.47650206089019775
leaky3 dv:1
leaky3 du:[0.]
leaky3 vth:0.6436737179756165


/home/franzhd/miniconda3/envs/rwkv-lava/src/lava/src/lava/magma/compiler/compiler_graphs.py:895: UserWarning: Cannot import module '<module 'dataloader' from '/home/franzhd/lava_AHPC/experiments/../src/utils/dataloader.py'>' when searching ProcessModels for Process 'OutputProcess'.
  warnings.warn(
/home/franzhd/miniconda3/envs/rwkv-lava/src/lava/src/lava/magma/compiler/compiler_graphs.py:895: UserWarning: Cannot import module '<module 'dataloader' from '/home/franzhd/lava_AHPC/experiments/../src/utils/dataloader.py'>' when searching ProcessModels for Process 'WISDM_spiking_dataloader'.
  warnings.warn(
100%|██████████| 200/200 [00:34<00:00,  5.87it/s]


Ground truth: [6 2 6 2 6 2 1 5 5 5 0 1 4 2 4 3 3 5 1 3 4 5 0 5 4 1 0 5 0 5 3 3 2 3 3 2 6 1 3 5 3 2 3 5 6 1 2 6 3 6 6 4 1 2 0 1 3 2 4 5 5 2 5 3 4 4 2 5 3 4 4 2 2 1 3 6 2 0 3 1 4 2 1 4 3 6 6 3 4 5 3 4 1 4 6 1 4 6 5 1 2 0 2 3 3 2 3 0 5 6 3 3 5 4 1 6 6 3 1 3 1 5 0 4 0 0 4 1 2 3 2 1 4 4 6 6 4 1 1 4 1 5 0 4 3 6 4 5 2 2 5 2 6 2 2 1 2 6 5 6 0 1 2 4 6 0 5 5 2 2 6 4 5 1 6 2 4 1 0 2 6 0 4 5 3 1 3 2 2 1 2 1 2 3 4 2 4 5 6]
Predictions : [0 2 5 3 5 2 0 0 0 5 2 2 0 2 1 5 0 2 2 1 0 0 5 1 1 0 0 0 0 1 1 3 2 3 1 2 6 0 2 5 0 2 3 1 6 0 2 6 0 1 1 5 0 6 0 0 1 2 1 6 0 0 5 0 3 0 2 6 1 1 0 2 2 0 1 2 5 2 0 1 0 1 2 0 0 5 0 3 0 2 3 0 2 0 6 5 0 6 2 0 2 0 0 0 2 2 3 2 2 5 2 3 6 2 0 1 6 3 0 3 1 6 0 2 0 0 2 1 2 3 2 1 4 0 0 0 0 5 1 0 0 2 2 1 1 4 0 0 2 2 5 2 0 2 2 2 2 1 5 1 2 0 0 0 4 0 5 5 5 2 1 3 2 0 0 2 0 0 0 2 0 2 0 0 2 5 4 5 2 1 2 0 2 0 4 2 0 0 1]
Accuracy    : 34.0


In [11]:
accuracy = np.sum(ground_truth==predictions)/ground_truth.size * 100

print(f"\nGround truth: {ground_truth}\n"
      f"Predictions : {predictions}\n"
      f"Accuracy    : {accuracy}")



Ground truth: [6 2 6 2 6 2 1 5 5 5 0 1 4 2 4 3 3 5 1 3 4 5 0 5 4 1 0 5 0 5 3 3 2 3 3 2 6 1 3 5 3 2 3 5 6 1 2 6 3 6 6 4 1 2 0 1 3 2 4 5 5 2 5 3 4 4 2 5 3 4 4 2 2 1 3 6 2 0 3 1 4 2 1 4 3 6 6 3 4 5 3 4 1 4 6 1 4 6 5 1 2 0 2 3 3 2 3 0 5 6 3 3 5 4 1 6 6 3 1 3 1 5 0 4 0 0 4 1 2 3 2 1 4 4 6 6 4 1 1 4 1 5 0 4 3 6 4 5 2 2 5 2 6 2 2 1 2 6 5 6 0 1 2 4 6 0 5 5 2 2 6 4 5 1 6 2 4 1 0 2 6 0 4 5 3 1 3 2 2 1 2 1 2 3 4 2 4 5 6 1]
Predictions : [0 0 2 5 3 5 2 0 0 0 5 2 2 0 2 1 5 0 2 2 1 0 0 5 1 1 0 0 0 0 1 1 3 2 3 1 2 6 0 2 5 0 2 3 1 6 0 2 6 0 1 1 5 0 6 0 0 1 2 1 6 0 0 5 0 3 0 2 6 1 1 0 2 2 0 1 2 5 2 0 1 0 1 2 0 0 5 0 3 0 2 3 0 2 0 6 5 0 6 2 0 2 0 0 0 2 2 3 2 2 5 2 3 6 2 0 1 6 3 0 3 1 6 0 2 0 0 2 1 2 3 2 1 4 0 0 0 0 5 1 0 0 2 2 1 1 4 0 0 2 2 5 2 0 2 2 2 2 1 5 1 2 0 0 0 4 0 5 5 5 2 1 3 2 0 0 2 0 0 0 2 0 2 0 0 2 5 4 5 2 1 2 0 2 0 4 2 0 0 1]
Accuracy    : 10.0
